In [66]:
def filter_by_pca_curvature(X, k=20, keep_ratio=0.3):
    tree = cKDTree(X) # Construct cKDTree from point cloud
    n = X.shape[0] # n is the number of points
    curvature_scores = np.zeros(n) # Initialize matrix of zeroes
    
    for i in range(n):
        _, idxs = tree.query(X[i], k=k+1)  # Include the point itself
        neighbors = X[idxs[1:]]  # Exclude the point itself, construct neighbors matrix
        cov = np.cov(neighbors - neighbors.mean(axis=0), rowvar=False) # Construct covariance matrix of centered neighbors
        eigvals = np.linalg.eigvalsh(cov) # Compute eigenvalues of the symmetric matrix
        eigvals = np.maximum(eigvals, 1e-10)  # Numerical stability
        #curvature_scores[i] = eigvals[0] / np.sum(eigvals)  # λ₃ / sum(λ)
        #curvature_scores[i] = eigvals[0]
        #curvature_scores[i] = (eigvals[1] - eigvals[0]) / eigvals[2]
        #curvature_scores[i] = eigvals[0] / np.sum(eigvals)  # as is
        curvature_scores[i] = (eigvals[2] - eigvals[0]) / eigvals[2] # Largest - smallest / largest

    # Keep top N% with highest curvature
    threshold_idx = int(n * keep_ratio) # Remove flat or nearly spherical ellipsoids of inertia
    keep_indices = np.argsort(curvature_scores)[-threshold_idx:]
    return X[keep_indices]

    # Sinkhorn
def sinkhorn(K, num_iters=100, tol=1e-6):
    u = np.ones(K.shape[0]) # Initalize flat array of n 1's
    v = np.ones(K.shape[1]) # Initalize flat array of n 1's
    for i in range(num_iters): # Loop through iterative convergence, as shown in the latex pdf
        u_prev = u.copy()
        Kv = K @ v
        u = 1.0 / np.maximum(Kv, 1e-8)
        KTu = K.T @ u
        v = 1.0 / np.maximum(KTu, 1e-8)
        delta = np.linalg.norm(u - u_prev) # compute delta, to track changes in u and convergence speed
        #print(f"Iter {i}, delta={delta:.2e}")
        if np.linalg.norm(u - u_prev) < tol: # Early stopping at convergence
            print(f"Sinkhorn converged at iteration {_}")
            break
            
    return np.diag(u) @ K @ np.diag(v)  # A: soft permutation matrix, project the 1D arrays of u and v on a diagonal


def random_orthogonal_matrix(d=3):
    H = np.random.randn(d, d) # Generate random orthogonal matrix
    Q, _ = np.linalg.qr(H) # Find orthonormal matrix using gram-schmidt (from QR decomposition)
    return Q

def reflection_group(d=3):
    return [np.diag(signs) for signs in product([-1, 1], repeat=d)] # Reflect the ellipsoid in 2^3 possible orientations


In [64]:
import numpy as np
from itertools import product
from scipy.optimize import linear_sum_assignment
import open3d as o3d
from scipy.spatial import cKDTree

np.random.seed()

# -------------------------------
# STEP 1: Load and Subsample Data
# -------------------------------
pcd = o3d.io.read_point_cloud("/Users/judahlevin/UATX/Linear Algebra/Final Project/Models/bunny.ply")
X_full = np.asarray(pcd.points)
n_full = X_full.shape[0]

n = 1000 # Set a number of points to sample from the point cloud
indices = np.random.choice(n_full, n, replace=False) # Randomly select
X_sampled = X_full[indices]
X = X_sampled # Set X to be the randomly n-sampled point cloud

# -------------------------------
# STEP 2: Generate Ground Truth Rotation + Permutation on Raw Cloud
# -------------------------------
# Save original subsampled point cloud for true evaluation
P_full = X - X.mean(axis=0)  # Center the n-sample point cloud before filtering
O = random_orthogonal_matrix()  # Generate random orthogonal matrix
perm = np.random.permutation(n)  # Generate random permutation matrix
Π_true = np.eye(n)[perm]  # Reorder the rows of an identity matrix of filtered dimension (1's and 0's)
P_perm = Π_true @ P_full  # Permute the n-sample (unfiltered) point cloud
Q_true = (O @ P_perm.T).T  # Compute n-sample ground truth with rigid transformation

# After building the full Q_true, apply the same filtering function to Q_true
Q_filtered = filter_by_pca_curvature(Q_true, k=20, keep_ratio=0.3)
Q = Q_filtered - Q_filtered.mean(axis=0)

# -------------------------------
# STEP 3: Filtered Point Cloud for Matching
# -------------------------------
X_filtered = filter_by_pca_curvature(X, k=20, keep_ratio=0.3)
P = X_filtered - X_filtered.mean(axis=0)  # Filtered + centered
n = P.shape[0]

# -------------------------------
# STEP 4: Kolpakov's E–Init + Sinkhorn + Hungarian
# -------------------------------
E_P = P.T @ P # Construct dxd covariance matrices with point clouds
E_Q = Q.T @ Q 
_, U_P = np.linalg.eigh(E_P) # Compute eigenvectors of the covariance matrices
_, U_Q = np.linalg.eigh(E_Q)
U0 = U_Q @ U_P.T  # Approximate initial alignment

U_best = None # Initialize best rotation
best_error = float("inf") # Initialize worst case error for later updates
col_ind_best = None # Initialize best permutation matrix
#epsilon = 0.1  # Deterministic entropic regularization

for D in reflection_group():
    U_candidate = U0 @ U_P @ D @ U_P.T # Compute candidate rotation matrix
    Q_candidate = Q_filtered @ U_candidate.T

    # Pairwise squared Euclidean distance matrix
    Q_sq = np.sum(Q_candidate**2, axis=1, keepdims=True)
    P_sq = np.sum(P**2, axis=1, keepdims=True)
    cost_matrix = Q_sq + P_sq.T - 2 * Q_candidate @ P.T # Compute l-2 squard norm by norm expansion 
    cost_std = np.std(cost_matrix) # Calculate standard deviation to measure cost matrix spread
    epsilon = 0.05 * cost_std # Compute dynamic entropic regularization with small value .05


    # Compute soft assignment matrix A
    K = np.exp(-(cost_matrix - np.min(cost_matrix)) / epsilon) # Compute K to solve the optimal transport problem
    A = sinkhorn(K) # Converge sinkhorn function to the optimal soft permutation matrix

    # Solve hard assignment using Hungarian
    row_ind, col_ind = linear_sum_assignment(-A) # Taking min of the inverse or negative = max, compute indices
    Π_est = np.zeros_like(A) # Initialize zero matrix of size nxn
    Π_est[row_ind, col_ind] = 1 # Sets entries of the index pairs to 1
    P_matched = P[col_ind] # Match P using the column/point permuting

    error = np.linalg.norm(P_matched - Q_candidate) # Compute frobenius norm, = np.sqrt(np.sum((P_matched - Q_candidate)**2))
    if error < best_error: # Update current best error, rotation matrix, and point permutation
        best_error = error
        U_best = U_candidate
        col_ind_best = col_ind

# -------------------------------
# STEP 5: Evaluation on Raw Cloud
# -------------------------------
δ_o = np.linalg.norm(U_best - O) # Compute rotation error

# -------------------------------
# Output Results
# -------------------------------
print("\n=== Kolpakov E–Init + Sinkhorn + Hungarian Evaluation ===")
print(f"δ_o   (rotation error):       {δ_o:.6f}")


=== Kolpakov E–Init + Sinkhorn + Hungarian Evaluation ===
δ_o   (rotation error):       2.828427
